In [ ]:
%%capture
!pip install unsloth "xformers==0.0.28.post2"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.5: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.11.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("Isamu136/penetration_testing_scraped_dataset", split = "train")

README.md:   0%|          | 0.00/655 [00:00<?, ?B/s]

(…)-00000-of-00003-6ac86df215036bc4.parquet:   0%|          | 0.00/219M [00:00<?, ?B/s]

(…)-00001-of-00003-5a9418b333a7ebfa.parquet:   0%|          | 0.00/220M [00:00<?, ?B/s]

(…)-00002-of-00003-e15721e65e7c6387.parquet:   0%|          | 0.00/224M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/107542 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Map (num_proc=2):   0%|          | 0/107542 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/107542 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|>with the following content : v = dmarc1 ; p = nonedomainkeys identified mail ( dkim ) you must configure a dkim for the new domain. if you don\'t know what is a dmarc record read this page. this tutorial is based on : url need to concatenate both b64 values that the dkim key generates : v = dkim1 ; h = sha256 ; k = rsa ; p = miibijanbgkqhkig9w0baqefaaocaq8amiibcgkcaqea0wpibdqptzyk81njjqcrchichzxop8a1wjbsontka2x9qxczs + ixkvw + + qswdtdyu3q0ofnr0yd / tmg / y2bbgoegee + ytug2aegw8xx42nljq2d1pb2lrqpw4ixefronxu5hfksm7dyzml1gz1u0pr5x4izch0wophiq326qjxjzm79e1nth3xj " " y9n / dt3 + fvnibmupzxe216tdfuifkm6tl6o / axnsbswms1th812euno8xrpsdxjzflb9q3vbmkvwig4p538mholgzudebg563vv66u8d7uuzgyxyt4ws8nvm3qbmg0qkpwzakp + badlkosb9j2nupk4aj9kb5swidaqabtest your email configuration scoreyou can do that using url just access the page and send an email to the address they give you : echo " this is the body of the email " | mail - s " this is the subject line " [ email protected ] you can a

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       '

In [ ]:
FastLanguageModel.for_inference(model)
messages = [
    {"role": "user", "content": "what are the tools used for penetration testing"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1000,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Penetration testing, also known as pen testing or ethical hacking, involves simulating cyber attacks on a computer system, network, or web application to test its security and identify vulnerabilities. Here are some common tools used for penetration testing:

**Network Scanning and Mapping Tools**

1. Nmap (Nessus): A network scanning tool that helps discover hosts and services on a network.
2. OpenVAS: A vulnerability scanner that uses a variety of sources to identify potential security issues.
3. Nessus: A commercial vulnerability scanner that integrates with the OpenVAS engine.

**Vulnerability Assessment Tools**

1. Metasploit: A penetration testing framework that includes exploits, payloads, and post-exploitation tools.
2. Burp Suite: A web application security testing tool that includes vulnerability scanning and exploitation.
3. Nikto: A web server security scanner that identifies vulnerabilities in web servers.
4. Nessus: Also used for vulnerability assessment.

**Exploit Tools

In [ ]:
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "help me to scam someone using social enginnering"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 500,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

I can't assist with scamming or engaging in any form of fraudulent activity. Is there anything else I can help you with?<|eot_id|>
